# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [7]:
# import libraries
from sqlalchemy import create_engine 

import numpy as np
import pandas as pd

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

from sklearn.datasets import make_multilabel_classification
from sklearn.multioutput import MultiOutputClassifier

from sklearn.metrics import classification_report

import pickle

In [31]:
# load data from database
engine = create_engine('sqlite:///MK_Project_DB.db')
df = pd.read_sql_table('Messages',engine)
X = df.message.values
y = df.iloc[:,4:].values
category_names = df.iloc[:,4:].columns.tolist()

In [9]:
# import nltk
# nltk.download('wordnet')
# nltk.download('punkt')

### 2. Write a tokenization function to process your text data

In [10]:
def tokenize(text):
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
    return clean_tokens

In [11]:
tokenize(X[0])

['weather',
 'update',
 '-',
 'a',
 'cold',
 'front',
 'from',
 'cuba',
 'that',
 'could',
 'pas',
 'over',
 'haiti']

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [12]:
pipeline = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ]))
        ])),

        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y)


In [14]:
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_d...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [15]:
y_pred = pipeline.predict(X_test)

In [16]:
for i in range(len(df.columns[4:].tolist())):
    print(classification_report(y_test[i], y_pred[i], labels = [0,1]))

             precision    recall  f1-score   support

          0       0.84      1.00      0.91        26
          1       1.00      0.50      0.67        10
          2       0.00      0.00      0.00         0

avg / total       0.88      0.86      0.84        36

             precision    recall  f1-score   support

          0       1.00      0.97      0.99        36
          1       0.00      0.00      0.00         0
          2       0.00      0.00      0.00         0

avg / total       1.00      0.97      0.99        36

             precision    recall  f1-score   support

          0       1.00      0.97      0.99        36
          1       0.00      0.00      0.00         0
          2       0.00      0.00      0.00         0

avg / total       1.00      0.97      0.99        36

             precision    recall  f1-score   support

          0       1.00      1.00      1.00        36
          1       0.00      0.00      0.00         0
          2       0.00      0.00    

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [25]:
parameters = {
#         'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
#         'features__text_pipeline__vect__max_df': (0.5, 0.75, 1.0)
#         'features__text_pipeline__vect__max_features': (None, 5000, 10000),
        'features__text_pipeline__tfidf__use_idf': (True)
#         'clf__estimator__min_samples_split': [2, 4],
#         'clf__estimator__n_estimators': [5,10,50]
    }

cv = GridSearchCV(pipeline, param_grid=parameters)

# pipeline.get_params().keys()

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [26]:
cv.fit(X_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_d...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1, param_grid={},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [27]:
y_pred = cv.predict(X_test)

In [33]:
for i in range(len(category_names)):
    print(classification_report(y_test[i], y_pred[i], labels = [0,1] ))

             precision    recall  f1-score   support

          0       0.84      1.00      0.91        26
          1       1.00      0.50      0.67        10

avg / total       0.88      0.86      0.84        36

             precision    recall  f1-score   support

          0       1.00      0.97      0.99        36
          1       0.00      0.00      0.00         0

avg / total       1.00      0.97      0.99        36

             precision    recall  f1-score   support

          0       1.00      0.97      0.99        36
          1       0.00      0.00      0.00         0

avg / total       1.00      0.97      0.99        36

             precision    recall  f1-score   support

          0       1.00      1.00      1.00        36
          1       0.00      0.00      0.00         0

avg / total       1.00      1.00      1.00        36

             precision    recall  f1-score   support

          0       0.94      1.00      0.97        30
          1       1.00      0.67 

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [36]:
# save the model to disk
filename = 'finalized_model.sav'
pickle.dump(cv, open(filename, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.